In [211]:
import os
from os import environ
from selenium import webdriver
from selenium.webdriver.support.select import Select
import dateparser
import datetime
import time
from selenium.webdriver.common.by import By 
from ipywidgets import FloatProgress
from IPython.display import display

In [212]:
first_name = 'First'
last_name = 'Last'
birth_date = 'MMDDYYYY'
permit_number = 'Y-------'
phone_number = '6280000000'

In [213]:
office_names = ['PLEASANTON',
                'OAKLAND CLAREMONT',
                'OAKLAND COLISEUM',
                'FREMONT',
                'DALY CITY',
                'HAYWARD',
                'SAN FRANCISCO',
                'EL CERRITO',
                'CONCORD',
                'WALNUT CREEK',
                'PITTSBURG']

In [214]:
driver = webdriver.Chrome()
driver.get('https://www.dmv.ca.gov/wasapp/foa/clear.do?goTo=driveTest&localeName=en')

In [215]:
def get_earliest_appointment_time(driver, office_name):
    time_slept = 0
    while len(driver.find_elements(By.ID, 'officeId'))==0 and time_slept < 10:
        time.sleep(1)
        time_slept += 1

    selector_field = driver.find_element_by_id('officeId')
    Select(selector_field).select_by_visible_text(office_name)

    test_type_selector = driver.find_element_by_id('DT')
    test_type_selector.click()
    
    first_name_input = driver.find_element_by_id('first_name')
    first_name_input.clear()
    first_name_input.send_keys(first_name)

    last_name_input = driver.find_element_by_id('last_name')
    last_name_input.clear()
    last_name_input.send_keys(last_name)

    permit_number_input = driver.find_element_by_id('dl_number')
    permit_number_input.clear()
    permit_number_input.send_keys(permit_number)

    driver.find_element_by_id('birthMonth').clear()
    driver.find_element_by_id('birthDay').clear()
    driver.find_element_by_id('birthYear').clear()
    driver.find_element_by_id('birthMonth').send_keys(birth_date)

    driver.find_element_by_id('areaCode').clear()
    driver.find_element_by_id('telPrefix').clear()
    driver.find_element_by_id('telSuffix').clear()
    driver.find_element_by_id('areaCode').send_keys(phone_number)

    submit = driver.find_element_by_class_name('btn-primary')
    submit.click()

    table_rows = driver.find_elements_by_css_selector('tbody > tr > td > p.no-margin-bottom > strong')
    earliest_date_str = table_rows[0].text
    earliest_date = dateparser.parse(earliest_date_str)
    driver.back()

    return earliest_date;

In [216]:
dates = {}
f = FloatProgress(min=0, max=len(office_names))
display(f)
for office_name in office_names:
    dates[office_name] = get_earliest_appointment_time(driver, office_name)
    f.value += 1

offices_by_time = sorted([date.isoformat() + ' ' + address for address, date in dates.items()])
offices_by_time

FloatProgress(value=0.0, max=11.0)

['2018-06-14T13:20:00 SAN FRANCISCO',
 '2018-06-19T09:40:00 PITTSBURG',
 '2018-06-20T13:20:00 OAKLAND COLISEUM',
 '2018-06-20T15:00:00 WALNUT CREEK',
 '2018-06-21T14:00:00 HAYWARD',
 '2018-06-25T08:00:00 OAKLAND CLAREMONT',
 '2018-06-25T09:20:00 CONCORD',
 '2018-06-25T11:20:00 EL CERRITO',
 '2018-06-25T14:40:00 PLEASANTON',
 '2018-07-05T08:20:00 FREMONT',
 '2018-07-27T09:20:00 DALY CITY']

['2018-05-29T08:40:00 PITTSBURG',
 '2018-06-12T15:20:00 SAN FRANCISCO',
 '2018-06-18T10:40:00 OAKLAND COLISEUM',
 '2018-06-18T14:40:00 HAYWARD',
 '2018-06-20T11:20:00 WALNUT CREEK',
 '2018-06-21T12:40:00 OAKLAND CLAREMONT',
 '2018-06-22T09:00:00 CONCORD',
 '2018-06-22T09:20:00 EL CERRITO',
 '2018-06-22T14:00:00 PLEASANTON',
 '2018-07-03T09:00:00 FREMONT',
 '2018-07-24T15:20:00 DALY CITY']